In [1]:
import chipwhisperer as cw
scope = cw.scope()

In [2]:
target = cw.target(scope, cw.targets.CW305, fpga_id="35t", force=False)

In [3]:
target.vccint_set(1.0)

In [4]:
target.pll.pll_enable_set(True)
target.pll.pll_outsource_set('PLL0', 0)

In [5]:
target.pll.pll_outfreq_set(freq=10E6, outnum=0)

In [6]:
print(target.pll.pll_outfreq_get(0))

10000000.0


In [7]:
scope.gain.db = 25
scope.adc.samples = 129
scope.adc.offset = 0
scope.adc.basic_mode = 'rising_edge'
scope.clock.adc_src = 'extclk_x4'
scope.io.hs2 = "disabled"

In [8]:
scope.clock.reset_adc()
assert (scope.clock.adc_locked == True)

In [9]:
proj = cw.create_project("passionayui.cwp", overwrite=True)

In [10]:
from tqdm.notebook import trange
N_traces = 5000
ktp = cw.ktp.Basic()
ktp.fixed_key = False
for k in range(10):
    key = ktp.next()[0]

for i in trange(N_traces):
    text = ktp.next()[1]
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        print("Timeout occured!!")
        continue
    proj.traces.append(trace)
proj.save()


  0%|          | 0/5000 [00:00<?, ?it/s]

In [11]:
scope.dis()
target.dis()

In [1]:
import chipwhisperer as cw
proj = cw.open_project("passionayui.cwp")

In [ ]:
import chipwhisperer.analyzer as cwa 
leak_model = cwa.leakage_models.last_round_state_diff
attack = cwa.cpa(proj, leak_model)
results = attack.run(cwa.get_jupyter_callback(attack), update_interval=1000)

In [2]:
import chipwhisperer.common.api.lascar as cw_lascar
from lascar import *
cw_container = cw_lascar.CWContainer(proj, proj.textouts)
guess_range = range(256)
leakage = cw_lascar.lastround_HD_gen
cpa_engines = [CpaEngine("cpa_%02d" % i, leakage(i), guess_range) for i in range(16)]
session = Session(cw_container, engines=cpa_engines).run(batch_size=1000)

rec_key_10r = []
for i in range(16):
    results = cpa_engines[i].finalize()
    guess = abs(results).max(1).argmax()
    rec_key_10r.append(guess)



2022-01-04 17:10:21,251 - lascar.session - INFO - Session Session: 5000 traces, 18 engines, batch_size=1000, leakage_shape=(129,)
INFO:lascar.session:Session Session: 5000 traces, 18 engines, batch_size=1000, leakage_shape=(129,)
Session |  0%||0 trc/5000 | (18 engines, batch_size=1000, leakage_shape=(129,)) |ETA:  --:--:--

Debug:
1000


Session |100%||5000 trc/5000 | (18 engines, batch_size=1000, leakage_shape=(129,)) |Time:  0:00:16


In [4]:
from chipwhisperer.analyzer.utils.aes_funcs import key_schedule_rounds
# rec_key_10r = results.key_guess() uncomment this if you use cwa.leakage_models instead of LASCAR
rec_key = key_schedule_rounds(rec_key_10r, 10, 0)
rec_key = bytearray(rec_key)
assert (rec_key == bytearray(proj.keys[0]))